In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import data
import models
import os
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


def valid(model, valid_loader,criterion):
    model.train(False)
    total_loss = 0.0
    total_correct = 0
    numpy_array = np.ones(2048,dtype='float32')
    label_array = np.ones(1,dtype='int')
    label_pred = np.ones(1,dtype='int')
    for inputs, labels in valid_loader:
        # stack X
        inputs = inputs.to(device)
        # this_numpy = myresnet.myforward(inputs).cpu().detach().numpy()
        # this_numpy = this_numpy.reshape(1,-1)
        # numpy_array = np.vstack([numpy_array,np.squeeze(this_numpy)])
        # print('X shape is:', numpy_array.shape)
        # stack y
        this_label = labels.numpy()
        label_array = np.vstack([label_array,(this_label)[:,None]])
        labels = labels.to(device)
        # calculate acc on valid set
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, predictions = torch.max(outputs, 1)
        total_loss += loss.item() * inputs.size(0)
        total_correct += torch.sum(predictions == labels.data)
        # print(predictions)
        label_pred = np.vstack([label_pred,(predictions.cpu().numpy())[:,None]])
        # print(label_pred.shape)
    epoch_loss = total_loss / len(valid_loader.dataset)
    epoch_acc = total_correct.double() / len(valid_loader.dataset)
    numpy_array = numpy_array[1:numpy_array.size]
    label_array = label_array[1:label_array.size]
    label_pred = label_pred[1:label_pred.size]
    return epoch_loss, epoch_acc.item(), numpy_array, label_array, label_pred


In [5]:

    model = torch.load('./models/best_modelC_myRenext18.pt')
    data_dir = "../hw2_dataset/"
    inupt_size = 224
    batch_size = 36
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    # myresnet = CustomResnet(model)

    criterion = nn.CrossEntropyLoss()
    train_loader, valid_loader = data.load_data(data_dir=data_dir,input_size=inupt_size, batch_size=batch_size)
    valid_loss, valid_acc, X, y, y_pred = valid(model, valid_loader,criterion)
    print("validation: {:.4f}, {:.4f}".format(valid_loss, valid_acc))
    matrix = confusion_matrix(y.squeeze(1),y_pred.squeeze(1))
    print('Confusion Matrix is : \n {}'.format(matrix))


    

validation: 1.0658, 0.8006
Confusion Matrix is : 
 [[566   3   0   0   1   1   0   0   1   1]
 [  0 570   0   0   0   0   0   0   0   0]
 [ 11  12 559   1   1   0   4   3   0   0]
 [ 54   5   8 267  47   2  13  54  72   0]
 [  0   0   2   2 470   0   6  34   2   0]
 [128  60  18  10   0 109  34   6  64   0]
 [110   2  85   0  12   1 263  40   8   0]
 [  0   0   1   0   3   0   0 573   0   0]
 [ 51   7  16  13   3   1   3   4 409   0]
 [ 37   7   2   0   0   0   0   0  11 537]]
